In [1]:
%run scrape_ons.ipynb

metadata = scrape('https://www.ons.gov.uk/businessindustryandtrade/business/businessinnovation/datasets/' \
                  'foreigndirectinvestmentinvolvingukcompanies2013inwardtables')
metadata

{'title': 'Foreign direct investment involving UK companies: Inward tables',
 'releaseDate': datetime.date(2017, 12, 1),
 'mailto': 'mailto:fdi@ons.gov.uk',
 'fileURL': 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/business/businessinnovation/datasets/foreigndirectinvestmentinvolvingukcompanies2013inwardtables/current/annualforeigndirectinvestmentinward2016.xls',
 'about': 'Inward datasets including data for flows, positions and earnings.'}

In [2]:
sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)
inputFile = sourceFolder / 'data.xls'
response = session.get(metadata['fileURL'])
with open(inputFile, 'wb') as f:
  f.write(response.content)
sheets = {sheet.name: sheet for sheet in loadxlstabs(inputFile)}

Loading in/data.xls which has size 724992 bytes
Table names: ['Contents', '1.1', '1.2', '1.3', '2.1', '2.2', '2.3', '3.1', '3.2', '3.3', '4.1', '4.2', '4.3', 'Geography', 'SIC']


1.1 to 1.3 are summary tables of the same form

In [3]:
from IPython.display import display, HTML

def toint(s):
    try:
        return int(float(s))
    except:
        return None

summary_tables = {}
for summary in [sheets['1.1'], sheets['1.2'], sheets['1.3']]:
    years = summary.excel_ref('A4').fill(RIGHT).is_not_blank()
    components = summary.excel_ref('A4').fill(DOWN).is_not_blank()
    component_label = components.by_index(1)
    components = components - components.regex('^Total').fill(DOWN) - component_label
    obs = components.fill(RIGHT).is_not_blank()
    cs = ConversionSegment(
        obs, [
            HDim(components, 'Component', DIRECTLY, LEFT),
            HDim(years, 'Year', DIRECTLY, ABOVE)
        ]
    )
    table = cs.topandas()
    table['Year']  = table['Year'].map(toint)
    table['Value'] = table['OBS'].map(toint)
    table['Unit']  = '£ million'
    to_drop = ['OBS']
    if 'DATAMARKER' in table:
        to_drop.append('DATAMARKER')
    table.drop(columns=to_drop, inplace=True)
    summary_tables[component_label.value] = table
    display(HTML(f'<h2>{component_label.value}</h2>'))
    display(table)

,Component,Year,Value,Unit
0,Foreign companies’ share of UK subsidiaries' a...,2007,47521,£ million
1,Foreign companies’ share of UK subsidiaries' a...,2008,44050,£ million
2,Foreign companies’ share of UK subsidiaries' a...,2009,41061,£ million
3,Foreign companies’ share of UK subsidiaries' a...,2010,37679,£ million
4,Foreign companies’ share of UK subsidiaries' a...,2011,42878,£ million
5,Foreign companies’ share of UK subsidiaries' a...,2012,41516,£ million
6,Foreign companies’ share of UK subsidiaries' a...,2013,47560,£ million
7,Foreign companies’ share of UK subsidiaries' a...,2014,45074,£ million
8,Foreign companies’ share of UK subsidiaries' a...,2015,45651,£ million
9,Foreign companies’ share of UK subsidiaries' a...,2016,49029,£ million


,Component,Year,Value,Unit
0,Foreign companies’ share of UK companies’ shar...,2007,494638.0,£ million
1,Foreign companies’ share of UK companies’ shar...,2008,552981.0,£ million
2,Foreign companies’ share of UK companies’ shar...,2009,543846.0,£ million
3,Foreign companies’ share of UK companies’ shar...,2010,606659.0,£ million
4,Foreign companies’ share of UK companies’ shar...,2011,651758.0,£ million
5,Foreign companies’ share of UK companies’ shar...,2012,781127.0,£ million
6,Foreign companies’ share of UK companies’ shar...,2013,812739.0,£ million
7,Foreign companies’ share of UK companies’ shar...,2014,892250.0,£ million
8,Foreign companies’ share of UK companies’ shar...,2015,917519.0,£ million
9,Foreign companies’ share of UK companies’ shar...,2016,1092732.0,£ million


,Component,Year,Value,Unit
0,Foreign companies’ share of UK subsidiaries’ a...,2007,43324.0,£ million
1,Foreign companies’ share of UK subsidiaries’ a...,2008,34875.0,£ million
2,Foreign companies’ share of UK subsidiaries’ a...,2009,33310.0,£ million
3,Foreign companies’ share of UK subsidiaries’ a...,2010,30381.0,£ million
4,Foreign companies’ share of UK subsidiaries’ a...,2011,33784.0,£ million
5,Foreign companies’ share of UK subsidiaries’ a...,2012,35297.0,£ million
6,Foreign companies’ share of UK subsidiaries’ a...,2013,40756.0,£ million
7,Foreign companies’ share of UK subsidiaries’ a...,2014,41538.0,£ million
8,Foreign companies’ share of UK subsidiaries’ a...,2015,41335.0,£ million
9,Foreign companies’ share of UK subsidiaries’ a...,2016,45184.0,£ million


Sheet 2.1 breakdown by country/area

In [4]:
by_area = sheets['2.1']
years = by_area.excel_ref('A5').expand(RIGHT).is_not_blank().is_not_whitespace()
continents = by_area.excel_ref('A5').expand(DOWN).is_not_blank()
continents = continents - continents.regex('WORLD TOTAL').fill(DOWN)
areas = by_area.excel_ref('B5').expand(DOWN).is_not_blank().filter(lambda c: c.value.strip() != 'of which')
areas = areas - areas.regex('CENTRAL & EASTERN EUROPE').fill(DOWN)
countries = by_area.excel_ref('C5').expand(DOWN).is_not_blank().is_not_whitespace()
obs = years.fill(DOWN)
#obs = (continents | areas | countries).fill(RIGHT)
obs = obs & (continents | areas | countries).expand(RIGHT)
cs = ConversionSegment(
    obs, [
        HDim(years, 'Year', DIRECTLY, ABOVE),
        HDim(continents, 'Continent', CLOSEST, ABOVE),
        HDim(areas, 'Area', CLOSEST, ABOVE),
        HDim(countries, 'Country', DIRECTLY, LEFT)
    ]
)
savepreviewhtml(cs)

In [5]:
t_by_area = cs.topandas()
t_by_area

,OBS,DATAMARKER,Year,Continent,Area,Country
0,38565,NaN,2007.0,EUROPE,None,None
1,22115,NaN,2008.0,EUROPE,None,None
2,22584,NaN,2009.0,EUROPE,None,None
3,8957,NaN,2010.0,EUROPE,None,None
4,-28258,NaN,2011.0,EUROPE,None,None
5,22830,NaN,2012.0,EUROPE,None,None
6,9456,NaN,2013.0,EUROPE,None,None
7,-8,NaN,2014.0,EUROPE,None,None
8,-12305,NaN,2015.0,EUROPE,None,None
9,97807,NaN,2016.0,EUROPE,None,None
